# JEPPESEN PROJECT

### Parse data from NAVDATA_POINTS, NAVDATA_SEGMENTS and the .srad-file from Jeppesen
---
#### Creates the following variables:
* **segments** - list with elements: 
    * `{'airway': 'AWY M771', 'from': 'WPT 34BC', 'to': 'WPT DONDA'}`
* **possible_destinations** - dictionary with keys: 
    * `'WPT ABTUB': {'NDB DP', ... , 'WPT UDINO'}`
* **allowed_routes** - list of lists containing segments: 
    * `[{'from': 'WPT INTIK', 'to': 'WPT ASURI'}, ... , {'from': 'VOR NLG', 'to': 'WPT TAMOT'}]`
* **points** - dictionary with keys:
    * `'WPT 0000E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E000000'}`

In [19]:
from graph import Graph

point_path = "../data/NAVDATA_POINTS"
segment_path = "../data/NAVDATA_SEGMENTS"
srad_path = "../data/_CHINA_MANDATE_OVERFLIGHT.srad"

mapping = {"1": "WPT",
           "2": "VOR",
           "3": "NDB",
           "4": "DME",
           "5": "APT",
           "6": "RWY"}

pointType = {'V': 'VOR',
             'W': 'WPT',
             'N': 'NDB',
             'A': 'APT',
             'D': 'DME',
             'R': 'RWY'}

def readFileLines(path):
    with open(path,'r') as f:
        return f.readlines()
    
def readFile(path):
    with open(path,'r') as f:
        return f.read()

#------------------------------------------------#
#      Create list of all segments in China      #
#------------------------------------------------#
def getSegments():
    segment_lines = readFileLines(segment_path)
    segments = []
    for line in segment_lines:
        if line[15]=="Z" or line[29]=="Z":  # pick the segments with at least one node in China ("z")
            segments.append({
                'airway': 'AWY '+line[2:8].strip(),
                'from': mapping[line[21]]+" "+line[9:14].strip(), 
                'to': mapping[line[35]]+" "+line[23:28].strip()
            })
    return segments
        
#---------------------------------------------------------------------------#
#      Create dictionary with all possible destinations for each point      #
#---------------------------------------------------------------------------#
def getPossibleDestinations(segments):
    tuples = [(seg['from'], seg['to']) for seg in segments]
    possible_destinations = Graph(tuples, directed=True)._graph
    return possible_destinations

#-------------------------------------------------#
#      Create a list with all allowed routes      #
#-------------------------------------------------#
def getAllowedRoutes():
    # import from file
    allowed_routes = []
    sequences = readFile(srad_path)
    # Clean input:   | indentation     | ")) " before next "(Seq"                              | the first ~30 rows
    sequences = [seq.replace('   ','').replace(')) ',')') for seq in sequences.split('(Seq\n')][1:]
    sequences = [seq.split('\n')[:-1] for seq in sequences]  # remove empty "" element from each sequence
    for seq in sequences:
        tmp_seq = []
        for seg in seq:
            cols = seg.split(' ')
            tmp_seq.append({
                'airway': cols[6]+" "+cols[7],
                'from': cols[8]+" "+cols[9],
                'to': cols[12]+" "+cols[13]
            })
        allowed_routes.append(tmp_seq)
    return allowed_routes
    
#--------------------------------------------------------#
#      Create a dictionary with all points in China      #
#--------------------------------------------------------#
def getPoints(segments):
    # import from file
    pointlines = readFileLines(point_path)
    points = {}
    for line in pointlines:
        points[pointType[line[0]]+" "+line[2:7].strip()] = {
            'area': line[19:23], 
            'lat': line[34:40].strip(), 
            'lon': line[41:48]
        }
    # create a dictionary with all points in China from our segments
    china_points = []
    for seg in segments:
        china_points.extend([seg['from'],seg['to']])
    china_points = list(set(china_points))
    # delete all points that are not in China
    points_all = dict(points)
    for p in points_all:
        if p not in china_points:
            del points[p]
    return points

{'WPT 34BC': {'area': 'ZJSA', 'lat': 'N14340', 'lon': 'E111555'},
 'VOR ABK': {'area': 'UNKL', 'lat': 'N53447', 'lon': 'E091231'},
 'WPT ABTUB': {'area': 'ZSHA', 'lat': 'N36000', 'lon': 'E117221'},
 'WPT ABVAX': {'area': 'ZYSH', 'lat': 'N47276', 'lon': 'E120354'},
 'WPT ABVER': {'area': 'ZSHA', 'lat': 'N35427', 'lon': 'E120334'},
 'WPT ABVIL': {'area': 'ZSHA', 'lat': 'N29385', 'lon': 'E119189'},
 'WPT ADBAG': {'area': 'ZPKM', 'lat': 'N24133', 'lon': 'E105130'},
 'WPT ADBAS': {'area': 'ZSHA', 'lat': 'N30206', 'lon': 'E120160'},
 'WPT ADBIN': {'area': 'ZGZU', 'lat': 'N21581', 'lon': 'E112493'},
 'WPT ADGOL': {'area': 'ZSHA', 'lat': 'N31287', 'lon': 'E116507'},
 'WPT ADKUN': {'area': 'ZJSA', 'lat': 'N18385', 'lon': 'E108106'},
 'WPT ADMUX': {'area': 'ZWUQ', 'lat': 'N38526', 'lon': 'E090570'},
 'WPT ADNEN': {'area': 'ZLHW', 'lat': 'N35304', 'lon': 'E107412'},
 'WPT ADNUR': {'area': 'UHHH', 'lat': 'N42125', 'lon': 'E130482'},
 'WPT ADPET': {'area': 'ZWUQ', 'lat': 'N44251', 'lon': 'E089416'}

In [4]:
segments

[{'airway': 'AWY M771', 'from': 'WPT 34BC', 'to': 'WPT DONDA'},
 {'airway': 'AWY P865', 'from': 'VOR ABK', 'to': 'WPT GINOM'},
 {'airway': 'AWY R104', 'from': 'VOR ABK', 'to': 'WPT GINOM'},
 {'airway': 'AWY A593', 'from': 'WPT ABTUB', 'to': 'WPT DALIM'},
 {'airway': 'AWY A593', 'from': 'WPT ABTUB', 'to': 'WPT UDINO'},
 {'airway': 'AWY W568', 'from': 'WPT ABTUB', 'to': 'NDB DP'},
 {'airway': 'AWY A345', 'from': 'WPT ABVAX', 'to': 'VOR HLD'},
 {'airway': 'AWY A345', 'from': 'WPT ABVAX', 'to': 'WPT ELPUN'},
 {'airway': 'AWY A591', 'from': 'WPT ABVER', 'to': 'VOR XDX'},
 {'airway': 'AWY A591', 'from': 'WPT ABVER', 'to': 'WPT LATUX'},
 {'airway': 'AWY W172', 'from': 'WPT ABVER', 'to': 'WPT URBAD'},
 {'airway': 'AWY G204', 'from': 'WPT ABVIL', 'to': 'WPT IDBIN'},
 {'airway': 'AWY G204', 'from': 'WPT ABVIL', 'to': 'WPT ELNEX'},
 {'airway': 'AWY A599', 'from': 'WPT ADBAG', 'to': 'VOR LXI'},
 {'airway': 'AWY A599', 'from': 'WPT ADBAG', 'to': 'WPT MEPAN'},
 {'airway': 'AWY R471', 'from': 'WPT AD

In [2]:
possible_destinations

defaultdict(set,
            {'WPT 34BC': {'WPT DONDA'},
             'VOR ABK': {'WPT GINOM'},
             'WPT ABTUB': {'NDB DP', 'WPT DALIM', 'WPT UDINO'},
             'WPT ABVAX': {'VOR HLD', 'WPT ELPUN'},
             'WPT ABVER': {'VOR XDX', 'WPT LATUX', 'WPT URBAD'},
             'WPT ABVIL': {'WPT ELNEX', 'WPT IDBIN'},
             'WPT ADBAG': {'VOR LXI', 'WPT KATBO', 'WPT MEPAN'},
             'WPT ADBAS': {'VOR HGH', 'WPT KAKIS'},
             'WPT ADBIN': {'WPT BOKAT', 'WPT TOMUD'},
             'WPT ADGOL': {'WPT IKUBA', 'WPT KIKEG'},
             'WPT ADKUN': {'WPT URDIN', 'WPT XELOV'},
             'WPT ADMUX': {'WPT NOLEP', 'WPT TUSLI'},
             'WPT ADNEN': {'NDB HO', 'VOR QIY', 'WPT TEBIB'},
             'WPT ADPET': {'VOR FKG', 'VOR QTV', 'WPT DAKPA', 'WPT NIRAV'},
             'WPT AGANU': {'WPT AMUTA'},
             'WPT AGAVO': {'WPT DONVO', 'WPT GONAV', 'WPT MUDAL', 'WPT NONOS'},
             'WPT AGOXA': {'VOR FKG', 'VOR QTV'},
             'WPT AGPOR': {

In [3]:
allowed_routes

[[{'airway': 'AWY W32', 'from': 'WPT INTIK', 'to': 'WPT ASURI'},
  {'airway': 'AWY W32', 'from': 'WPT ASURI', 'to': 'WPT PADSA'},
  {'airway': 'AWY W32', 'from': 'WPT PADSA', 'to': 'VOR HET'},
  {'airway': 'AWY B208', 'from': 'VOR HET', 'to': 'WPT DOTOS'},
  {'airway': 'AWY B208', 'from': 'WPT DOTOS', 'to': 'WPT ISBAM'},
  {'airway': 'AWY B208', 'from': 'WPT ISBAM', 'to': 'WPT ATPOK'},
  {'airway': 'AWY B208', 'from': 'WPT ATPOK', 'to': 'WPT TODAM'},
  {'airway': 'AWY B208', 'from': 'WPT TODAM', 'to': 'WPT EKETA'},
  {'airway': 'AWY B208', 'from': 'WPT EKETA', 'to': 'VOR TYN'},
  {'airway': 'AWY B208', 'from': 'VOR TYN', 'to': 'WPT XIVEP'},
  {'airway': 'AWY B208', 'from': 'WPT XIVEP', 'to': 'WPT ANPIG'},
  {'airway': 'AWY B208', 'from': 'WPT ANPIG', 'to': 'WPT EGEBI'},
  {'airway': 'AWY B208', 'from': 'WPT EGEBI', 'to': 'NDB SQ'},
  {'airway': 'AWY B208', 'from': 'NDB SQ', 'to': 'WPT PADNO'},
  {'airway': 'AWY B208', 'from': 'WPT PADNO', 'to': 'WPT TAMIX'},
  {'airway': 'AWY B208', 'f

In [25]:
points

{'WPT 0000E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E000000'},
 'WPT 0001E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E001000'},
 'WPT 0002E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E002000'},
 'WPT 0002N': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'W002000'},
 'WPT 0003E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E003000'},
 'WPT 0004E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E004000'},
 'WPT 0004N': {'area': 'GOOO', 'lat': 'N00000', 'lon': 'W004000'},
 'WPT 0005E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E005000'},
 'WPT 0006E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E006000'},
 'WPT 0007E': {'area': 'FCCC', 'lat': 'N00000', 'lon': 'E007000'},
 'WPT 0008E': {'area': 'FCCC', 'lat': 'N00000', 'lon': 'E008000'},
 'WPT 0008N': {'area': 'GOOO', 'lat': 'N00000', 'lon': 'W008000'},
 'WPT 0009E': {'area': 'FCCC', 'lat': 'N00000', 'lon': 'E009000'},
 'WPT 0009N': {'area': 'GOOO', 'lat': 'N00000', 'lon': 'W009000'},
 'WPT 0010E': {'area': 'FCCC', 'lat': 'N00000', 'lon': 'E01000